# TSA Chapter 0: Exponential Smoothing Methods

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/QuantLet/TSA/blob/main/TSA_Ch0/TSA_ch0_smoothing/TSA_ch0_smoothing.ipynb)

This notebook demonstrates exponential smoothing methods:
- Simple Exponential Smoothing (SES)
- Holt's linear trend method
- Holt-Winters seasonal method
- The ETS (Error-Trend-Seasonality) framework

In [ ]:
!pip install matplotlib numpy scipy -q

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats

In [ ]:
# Style configuration
COLORS = {
    'blue': '#1A3A6E',
    'red': '#DC3545',
    'green': '#2E7D32',
    'orange': '#E67E22',
    'gray': '#666666',
    'purple': '#8E44AD',
}

plt.rcParams.update({
    'axes.facecolor': 'none',
    'figure.facecolor': 'none',
    'savefig.transparent': True,
    'axes.spines.top': False,
    'axes.spines.right': False,
    'font.size': 10,
    'axes.titlesize': 11,
    'axes.labelsize': 10,
    'legend.fontsize': 8,
    'xtick.labelsize': 8,
    'ytick.labelsize': 8,
    'lines.linewidth': 1.5,
    'axes.prop_cycle': plt.cycler('color', list(COLORS.values())),
    'axes.edgecolor': '#333333',
    'axes.linewidth': 0.8,
    'axes.grid': False,
})

np.random.seed(42)

CHARTS_DIR = os.path.join(os.path.dirname(os.path.abspath('.')), '..', '..', 'charts')

def save_chart(fig, name):
    fig.savefig(f'{name}.pdf', bbox_inches='tight', transparent=True, dpi=150)
    fig.savefig(f'{name}.png', bbox_inches='tight', transparent=True, dpi=150)
    # Also save to main charts directory for the lecture
    try:
        charts_path = os.path.join(CHARTS_DIR, name)
        fig.savefig(f'{charts_path}.pdf', bbox_inches='tight', transparent=True, dpi=150)
        fig.savefig(f'{charts_path}.png', bbox_inches='tight', transparent=True, dpi=150)
    except Exception:
        pass  # Skip if running on Colab without the charts dir
    print(f'Saved: {name}.pdf + .png')

def add_legend_below(ax, ncol=3):
    ax.legend(loc='upper center', bbox_to_anchor=(0.5, -0.12), ncol=ncol, frameon=False)

In [ ]:
# Chart: simple_exp_smoothing
# SES with different alpha values
np.random.seed(42)
fig, ax = plt.subplots(figsize=(8, 4.5))

t = np.arange(80)
y = 50 + np.cumsum(np.random.randn(80) * 1.5)

ax.plot(t, y, color=COLORS['gray'], linewidth=1.0, alpha=0.5, label='Original data')

for alpha, c, ls in [(0.1, COLORS['blue'], '-'), (0.5, COLORS['green'], '--'), (0.9, COLORS['red'], '-.')]:
    ses = np.zeros(80)
    ses[0] = y[0]
    for i in range(1, 80):
        ses[i] = alpha * y[i] + (1 - alpha) * ses[i-1]
    ax.plot(t, ses, color=c, linewidth=1.5, linestyle=ls, label=f'SES ($\\alpha={alpha}$)')

ax.set_title('Simple Exponential Smoothing (SES): Effect of $\\alpha$', fontweight='bold')
ax.set_xlabel('Time ($t$)')
ax.set_ylabel('$\\hat{X}_t$')
add_legend_below(ax, ncol=4)

fig.tight_layout()
save_chart(fig, 'simple_exp_smoothing')
plt.show()

In [ ]:
# Chart: holt_method
# Holt's linear trend method with forecast
np.random.seed(42)
fig, ax = plt.subplots(figsize=(8, 4.5))

t = np.arange(60)
y = 50 + 0.6 * t + np.random.randn(60) * 3

# Holt's method implementation
alpha, beta = 0.3, 0.1
level = np.zeros(60)
trend_comp = np.zeros(60)
level[0] = y[0]
trend_comp[0] = y[1] - y[0]

for i in range(1, 60):
    level[i] = alpha * y[i] + (1 - alpha) * (level[i-1] + trend_comp[i-1])
    trend_comp[i] = beta * (level[i] - level[i-1]) + (1 - beta) * trend_comp[i-1]

fitted = level + trend_comp

# Forecast
h_steps = 15
forecast = np.array([level[-1] + (h+1) * trend_comp[-1] for h in range(h_steps)])
t_f = np.arange(60, 60 + h_steps)
ci_width = np.linspace(4, 12, h_steps)

ax.plot(t, y, color=COLORS['gray'], linewidth=0.8, alpha=0.5, label='Original data')
ax.plot(t, fitted, color=COLORS['blue'], linewidth=1.5, label='Holt fitted')
ax.plot(t_f, forecast, color=COLORS['red'], linewidth=2, linestyle='--', label='Forecast')
ax.fill_between(t_f, forecast - ci_width, forecast + ci_width,
                color=COLORS['red'], alpha=0.15, label='95% CI')
ax.axvline(x=59.5, color=COLORS['gray'], linestyle=':', linewidth=0.8)

ax.set_title("Holt's Method: Level + Linear Trend", fontweight='bold')
ax.set_xlabel('Time ($t$)')
ax.set_ylabel('$X_t$')
add_legend_below(ax, ncol=4)

fig.tight_layout()
save_chart(fig, 'holt_method')
plt.show()

In [ ]:
# Chart: holt_winters
# Holt-Winters method with seasonal component and forecast
np.random.seed(42)
fig, ax = plt.subplots(figsize=(8, 4.5))

t = np.arange(72)
trend = 50 + 0.4 * t
seasonal = 12 * np.sin(2 * np.pi * t / 12)
y = trend + seasonal + np.random.randn(72) * 2

# HW fitted (simplified)
alpha, beta, gamma = 0.3, 0.1, 0.3
s = 12
level = np.zeros(72)
trend_c = np.zeros(72)
seas = np.zeros(72 + s)

# Initialize
level[0] = np.mean(y[:s])
trend_c[0] = (np.mean(y[s:2*s]) - np.mean(y[:s])) / s
for j in range(s):
    seas[j] = y[j] - level[0]

for i in range(1, 72):
    level[i] = alpha * (y[i] - seas[i - s + s]) + (1 - alpha) * (level[i-1] + trend_c[i-1])
    trend_c[i] = beta * (level[i] - level[i-1]) + (1 - beta) * trend_c[i-1]
    seas[i + s] = gamma * (y[i] - level[i]) + (1 - gamma) * seas[i]

fitted = level + trend_c + seas[s:s+72]

# Forecast
h_steps = 18
t_f = np.arange(72, 72 + h_steps)
forecast = np.array([level[-1] + (h+1) * trend_c[-1] + seas[72 + (h % s)] for h in range(h_steps)])
ci_width = np.linspace(3, 10, h_steps)

ax.plot(t, y, color=COLORS['gray'], linewidth=0.8, alpha=0.5, label='Original data')
ax.plot(t, fitted, color=COLORS['blue'], linewidth=1.5, label='Holt-Winters fitted')
ax.plot(t_f, forecast, color=COLORS['red'], linewidth=2, linestyle='--', label='Forecast')
ax.fill_between(t_f, forecast - ci_width, forecast + ci_width,
                color=COLORS['red'], alpha=0.15, label='95% CI')
ax.axvline(x=71.5, color=COLORS['gray'], linestyle=':', linewidth=0.8)

ax.set_title('Holt-Winters: Level + Trend + Seasonality', fontweight='bold')
ax.set_xlabel('Time (months)')
ax.set_ylabel('$X_t$')
add_legend_below(ax, ncol=4)

fig.tight_layout()
save_chart(fig, 'holt_winters')
plt.show()

In [ ]:
# Chart: ch1_def_ets
# Exponential weights and comparison of alpha values
np.random.seed(42)
fig, axes = plt.subplots(1, 2, figsize=(10, 4))

t = np.arange(20)
alpha = 0.3
weights = alpha * (1 - alpha) ** t
axes[0].bar(t, weights, color=COLORS['blue'], alpha=0.7, edgecolor='white')
axes[0].set_title('Exponential Weights ($\\alpha=0.3$)', fontweight='bold')
axes[0].set_xlabel('Lag ($k$)')
axes[0].set_ylabel('Weight: $\\alpha(1-\\alpha)^k$')

for alpha, c, ls in [(0.1, COLORS['blue'], '-'), (0.3, COLORS['green'], '--'),
                      (0.7, COLORS['orange'], '-.')]:
    w = alpha * (1 - alpha) ** t
    axes[1].plot(t, w, color=c, linewidth=1.5, linestyle=ls, marker='o', markersize=3,
                label=f'$\\alpha={alpha}$')

axes[1].set_title('Weights Comparison', fontweight='bold')
axes[1].set_xlabel('Lag ($k$)')
axes[1].set_ylabel('Weight')
add_legend_below(axes[1], ncol=3)

fig.tight_layout(rect=[0, 0.02, 1, 1])
save_chart(fig, 'ch1_def_ets')
plt.show()

In [ ]:
# Chart: ets_components
# ETS framework: different model types
np.random.seed(42)
fig, axes = plt.subplots(2, 2, figsize=(10, 5.5))

t = np.arange(72)

# ETS(A,N,N) - SES
y_ann = 50 + np.cumsum(np.random.randn(72) * 1.5)
axes[0, 0].plot(t, y_ann, color=COLORS['gray'], linewidth=0.8, alpha=0.5)
ses = np.zeros(72)
ses[0] = y_ann[0]
for i in range(1, 72):
    ses[i] = 0.3 * y_ann[i] + 0.7 * ses[i-1]
axes[0, 0].plot(t, ses, color=COLORS['blue'], linewidth=1.5)
axes[0, 0].set_title('ETS(A,N,N): Level only', fontweight='bold', fontsize=9)

# ETS(A,A,N) - Holt
y_aan = 50 + 0.5 * t + np.random.randn(72) * 3
axes[0, 1].plot(t, y_aan, color=COLORS['gray'], linewidth=0.8, alpha=0.5)
axes[0, 1].plot(t, 50 + 0.5 * t, color=COLORS['red'], linewidth=1.5)
axes[0, 1].set_title('ETS(A,A,N): Level + Trend', fontweight='bold', fontsize=9)

# ETS(A,A,A) - HW Additive
y_aaa = 50 + 0.3 * t + 10 * np.sin(2*np.pi*t/12) + np.random.randn(72) * 2
axes[1, 0].plot(t, y_aaa, color=COLORS['gray'], linewidth=0.8, alpha=0.5)
axes[1, 0].plot(t, 50 + 0.3 * t + 10 * np.sin(2*np.pi*t/12), color=COLORS['green'], linewidth=1.5)
axes[1, 0].set_title('ETS(A,A,A): Additive Holt-Winters', fontweight='bold', fontsize=9)
axes[1, 0].set_xlabel('Time')

# ETS(M,A,M) - HW Multiplicative
tr = 50 + 0.5 * t
y_mam = tr * (1 + 0.15 * np.sin(2*np.pi*t/12)) * (1 + np.random.randn(72)*0.02)
axes[1, 1].plot(t, y_mam, color=COLORS['gray'], linewidth=0.8, alpha=0.5)
axes[1, 1].plot(t, tr * (1 + 0.15 * np.sin(2*np.pi*t/12)), color=COLORS['orange'], linewidth=1.5)
axes[1, 1].set_title('ETS(M,A,M): Multiplicative Holt-Winters', fontweight='bold', fontsize=9)
axes[1, 1].set_xlabel('Time')

fig.suptitle('ETS Framework: Error-Trend-Seasonality', fontweight='bold', fontsize=12, y=1.02)
fig.tight_layout()
save_chart(fig, 'ets_components')
plt.show()